In [1]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
from tqdm import tqdm

/home/nagajas/.conda/envs/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class ChessDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_len=512):
        self.data = pd.read_csv(csv_file)
        self.tokenizer = tokenizer
        self.max_len = max_len

        self.data["Delta"] = self.data["Delta"].apply(
            lambda x: 0 if pd.isna(x) or x == "Mate" else x
        )
        self.data["Top K Best Moves"] = self.data["Top K Best Moves"].apply(
            lambda x: "No moves" if not isinstance(x, str) or not x.strip() else x
        )
        self.data["Commentary"] = self.data["Commentary"].apply(
            lambda x: "No commentary available" if not isinstance(x, str) or not x.strip() else x
        )
        self.data['Eval Before'] = pd.to_numeric(self.data['Eval Before'], errors='coerce').fillna(0)
        self.data['Eval After'] = pd.to_numeric(self.data['Eval After'], errors='coerce').fillna(0)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        input_text = (
            f"Chess Commentary Generation: "
            f"History: {row['History (PGN)']} | Current Move: {row['Move']} | "
            f"Evaluation Before: {row['Eval Before']} ({'Advantage to White' if row['Eval Before'] > 0 else 'Advantage to Black'}) | "
            f"Evaluation After: {row['Eval After']} ({'Improved' if row['Eval After'] > row['Eval Before'] else 'Declined'}) | "
            f"Delta: {row['Delta']} | Top Suggestions: {row['Top K Best Moves']}"
        )
        commentary = row["Commentary"]

        input_encoding = self.tokenizer(
            input_text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        target_encoding = self.tokenizer(
            commentary,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        return {
            "input_ids": input_encoding["input_ids"].squeeze(0),
            "attention_mask": input_encoding["attention_mask"].squeeze(0),
            "labels": target_encoding["input_ids"].squeeze(0),
        }


In [3]:
def train_and_evaluate(
    model, train_loader, val_loader, epochs, learning_rate, device, log_interval=100
):
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        model.train()
        total_train_loss = 0
        correct_train_predictions = 0
        total_train_samples = 0

        for step, batch in enumerate(tqdm(train_loader, desc="Training"), 1):
            optimizer.zero_grad()

            outputs = model(
                input_ids=batch["input_ids"].to(device),
                attention_mask=batch["attention_mask"].to(device),
                labels=batch["labels"].to(device),
            )
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            scheduler.step()

            total_train_loss += loss.item()

            # Calculate accuracy for training
            logits = outputs.logits.argmax(dim=-1)
            correct_train_predictions += (
                (logits == batch["labels"].to(device)).float().mean().item()
            )
            total_train_samples += 1

            if step % log_interval == 0:
                train_loss = total_train_loss / step
                train_accuracy = correct_train_predictions / total_train_samples
                val_loss, val_accuracy = evaluate_model(model, val_loader, device)

                print(
                    f"Step {step} | "
                    f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_accuracy:.4f} | "
                    f"Validation Loss: {val_loss:.4f} | Validation Accuracy: {val_accuracy:.4f}"
                )

        print(
            f"Epoch {epoch + 1} Summary: "
            f"Train Loss: {total_train_loss / len(train_loader):.4f} | "
            f"Train Accuracy: {correct_train_predictions / total_train_samples:.4f}"
        )

In [4]:
def evaluate_model(model, val_loader, device):
    model.eval()
    total_val_loss = 0
    correct_val_predictions = 0
    total_val_samples = 0

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation", leave=False):
            outputs = model(
                input_ids=batch["input_ids"].to(device),
                attention_mask=batch["attention_mask"].to(device),
                labels=batch["labels"].to(device),
            )
            loss = outputs.loss
            total_val_loss += loss.item()

            logits = outputs.logits.argmax(dim=-1)
            correct_val_predictions += (
                (logits == batch["labels"].to(device)).float().mean().item()
            )
            total_val_samples += 1

    avg_val_loss = total_val_loss / len(val_loader)
    avg_val_accuracy = correct_val_predictions / total_val_samples
    return avg_val_loss, avg_val_accuracy

In [6]:
%pip install sentencepiece

if __name__ == "__main__":
    TRAIN_FILE = "./data/train.csv"  
    VAL_FILE = "./data/val.csv" 
    MODEL_NAME = "t5-small" 
    MAX_LEN = 512
    BATCH_SIZE = 8  
    EPOCHS = 2
    LEARNING_RATE = 5e-4 
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
    model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
    model.to(DEVICE)

    train_dataset = ChessDataset(TRAIN_FILE, tokenizer, MAX_LEN)
    val_dataset = ChessDataset(VAL_FILE, tokenizer, MAX_LEN)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    train_and_evaluate(model, train_loader, val_loader, EPOCHS, LEARNING_RATE, DEVICE)

    model.save_pretrained("./chess_commentary_model")
    tokenizer.save_pretrained("./chess_commentary_model")
    print("Model and tokenizer saved.")

Note: you may need to restart the kernel to use updated packages.
Epoch 1/2


Training:  11%|█▏        | 100/875 [01:16<2:08:13,  9.93s/it]

Step 100 | Train Loss: 0.8170 | Train Accuracy: 0.9069 | Validation Loss: 0.3239 | Validation Accuracy: 0.9556


Training:  23%|██▎       | 200/875 [02:31<1:52:36, 10.01s/it]

Step 200 | Train Loss: 0.5775 | Train Accuracy: 0.9302 | Validation Loss: 0.3229 | Validation Accuracy: 0.9556


...

Step 700 | Train Loss: 0.3377 | Train Accuracy: 0.9533 | Validation Loss: 0.3228 | Validation Accuracy: 0.9556


Training:  91%|█████████▏| 800/875 [10:00<12:30, 10.01s/it]

Step 800 | Train Loss: 0.3373 | Train Accuracy: 0.9534 | Validation Loss: 0.3228 | Validation Accuracy: 0.9556


Training: 100%|██████████| 875/875 [10:32<00:00,  1.38it/s]


Epoch 2 Summary: Train Loss: 0.3354 | Train Accuracy: 0.9537
Model and tokenizer saved.
